In [14]:
from omegaconf import OmegaConf
cfg = OmegaConf.load("/work/users/t/i/tis/VidMuse/output/VidMuse/xps/31a205cf/.hydra/config.yaml")
cfg.dataset.batch_size = 2
cfg.dataset.num_workers = 0
print("Batch Size:", cfg.dataset.batch_size)
print("Num Worker:", cfg.dataset.num_workers)

Batch Size: 2
Num Worker: 0


In [15]:
print(OmegaConf.to_yaml(cfg, resolve=True))

device: cuda
dtype: float32
autocast: true
autocast_dtype: float16
seed: 2036
show: false
continue_from: //pretrained/facebook/musicgen-small
execute_only: null
execute_inplace: false
benchmark_no_load: false
efficient_attention_backend: torch
num_threads: 1
mp_start_method: forkserver
label: null
logging:
  level: INFO
  log_updates: 1
  log_tensorboard: true
  log_wandb: true
tensorboard:
  with_media_logging: false
  name: null
  sub_dir: null
wandb:
  with_media_logging: true
  project: VideoMusicGeneration
  name: default
  group: default
slurm:
  gpus: 2
  mem_per_gpu: 32
  time: 600
  constraint: null
  partition: l40-gpu
  comment: null
  setup:
  - source ~/.bashrc
  - echo "FFMPEG located at $(which ffmpeg)"
  - echo "$(ffmpeg -version)"
  - cd /work/users/t/i/tis/VidMuse
  - source .venv/bin/activate
  exclude: null
  qos: gpu_access
  cpus_per_gpu: 4
  one_task_per_node: false
dora:
  dir: /checkpoint/tis/experiments/audiocraft/outputs
  exclude:
  - device
  - wandb.*
  - 

In [2]:
from audiocraft.solvers.builders import get_audio_datasets, DatasetType
dataloaders = get_audio_datasets(cfg, dataset_type=DatasetType.MUSIC)
print(dataloaders.keys())

num_workers in builder: 0
Using 0 and 2 for data loading
num_workers in builder: 0
Using 0 and 2 for data loading
num_workers in builder: 0
Using 0 and 2 for data loading
num_workers in builder: 0
Using 0 and 2 for data loading
dict_keys(['train', 'valid', 'evaluate', 'generate'])


In [ ]:
#ld = dataloaders['train']
ld = dataloaders['valid']
it = iter(ld)
batch = next(it)
audio, video, info = batch

/work/users/t/i/tis/V2Music/preprocessing/data/video/jx0u5fNZECQ.mp4
fps=25.00, total_frames=751
video frames: 58
flow frames: 117
video_tensor shape torch.Size([3, 58, 224, 224])
flow_tensor shape torch.Size([1, 29, 224, 224])
loading from here
Called getitem of InfoAudioDataset
Called getitem in MusicDataset
/work/users/t/i/tis/V2Music/preprocessing/data/video/C0o1BhJJ1ZI.mp4
fps=25.00, total_frames=751
video frames: 58
flow frames: 117
video_tensor shape torch.Size([3, 58, 224, 224])
flow_tensor shape torch.Size([1, 29, 224, 224])
loading from here
Called getitem of InfoAudioDataset
Called getitem in MusicDataset


In [13]:
len(batch)

3

In [7]:
print(type(audio), type(video), type(info))
vids, flows = video
v_infos, f_infos = info
print(v_infos)
print(f_infos)
print("batch_size:", ld.batch_size)
print("num_workers:", ld.num_workers)
print("Audio Tensor", audio.shape)
print("Video Tensor", vids.shape)
print("Flow Tensor", flows.shape)

<class 'torch.Tensor'> <class 'list'> <class 'list'>
MusicInfo(meta=AudioMeta(path='/work/users/t/i/tis/V2Music/preprocessing/bg_audio/mdx_extra/jx0u5fNZECQ_no_vocals.wav', video_path='/work/users/t/i/tis/V2Music/preprocessing/data/video/jx0u5fNZECQ.mp4', duration=29.993514739229024, sample_rate=44100, amplitude=None, weight=None, info_path=None), seek_time=0, n_frames=928000, total_frames=928000, sample_rate=32000, channels=1, audio_tokens=None, title=None, artist=None, key=None, bpm=None, genre=None, moods=None, keywords=None, description=None, name=None, instrument=None, self_wav=WavCondition(wav=tensor([[[-0.0004, -0.0394, -0.0493,  ...,  0.0098,  0.0088,  0.0106]]]), length=tensor([928000]), sample_rate=[32000], path=['/work/users/t/i/tis/V2Music/preprocessing/bg_audio/mdx_extra/jx0u5fNZECQ_no_vocals.wav'], seek_time=[0]), joint_embed={})
MusicInfo(meta=AudioMeta(path='/work/users/t/i/tis/V2Music/preprocessing/bg_audio/mdx_extra/C0o1BhJJ1ZI_no_vocals.wav', video_path='/work/users/

In [4]:
from audiocraft.models.builders import get_lm_model
lm = get_lm_model(cfg)

/work/users/t/i/tis/VidMuse/.venv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [8]:
video_emb = lm.compute_video_emb(video, device=cfg.device)

Visual: torch.Size([2, 58, 768])
Flow: torch.Size([2, 29, 768])


In [7]:
video_emb.size()

torch.Size([2, 58, 1024])

In [10]:
from audiocraft.solvers import CompressionSolver
compression_model = CompressionSolver.wrapped_model_from_checkpoint(cfg, cfg.compression_model_checkpoint, device=cfg.device)

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/236M [00:00<?, ?B/s]

In [11]:
audio_tokens, scale = compression_model.encode(audio)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [9]:
lm.forward(audio, [], video)

AssertionError: Sequence shape must match the specified number of codebooks